In [1]:
from models.q_network import CardFeaturizer
import torch as t
from game.utils import variable
import numpy as np
from game.game_utils import Card, cards_to_array
from matplotlib import pyplot as plt
from IPython import display

# Load dataset

In [ ]:
import pickle
filename = 'file.pkl'
with open(filename, 'wb') as f:
    dataset = pickle.load(f)


# Parse dataset

frozenset({'2d', '6c', '9d', '7s'})): {'player1winprob': 0.15217391304347827, 'High Card': 0.5217391304347826, 'Pair': 0.391304347826087, 'Two Pair': 0.0, 'Three of a Kind': 0.0, 'Straight': 0.08695652173913043, 'Flush': 0.0, 'Full House': 0.0, 'Four of a Kind': 0.0, 'Straight Flush': 0.0, 'Royal Flush': 0.0}}
Time diff is 0.002004861831665039

In [ ]:
x_hand = np.zeros((len(dataset), 13, 4))
x_board = np.zeros((len(dataset), 3, 13, 4))
y = np.zeros((len(dataset), 10))

for i, ((hand_, board_), probas) in enumerate(dataset.items()):
    hand = []
    board = []
    for card_ in hand_:
        hand.append(Card(card[0], card[1]))
    for card_ in board_:
        board.append(Card(card[0], card[1]))
    x_hand[i] = cards_to_array(hand)
    x_board[i] = cards_to_array(board)
    y[i][0] = probas['player1winprob']
    y[i][1] = probas['High Card']
    y[i][2] = probas['Pair']
    y[i][3] = probas['Two Pair']
    y[i][4] = probas['Three of a Kind']
    y[i][5] = probas['Straight']
    y[i][6] = probas['Flush']
    y[i][7] = probas['Full House']
    y[i][8] = probas['Four of a Kind']
    y[i][9] = probas['Straight Flush']
    

# Train neural network

In [ ]:
f = CardFeaturizer(10)

In [ ]:
list(f.parameters())

In [ ]:
def moving_avg(x, window=50):
    return [np.mean(x[k:k+window]) for k in range(len(x)-window)]

In [ ]:
lr = 1e-3
batch_size = 64
EPOCHS = 50
optimizer = t.optim.Adam(f.parameters(), lr=lr)
losses = []
PLOT = 1000


for _ in range(EPOCHS):
    for i in range(0, len(dataset), batch_size):
        hand = variable(x_hand[i:i+batch_size])
        board = variable(x_board[i:i+batch_size])
        target = variable(y[i:i+batch_size])
        
        optimizer.zero_grad()

        output = f.forward(hand, board)
        
        loss = -t.log(output)*target
        loss = t.sum(loss)
        
        losses.append(loss.data.numpy()[0]/batch_size)
        
        loss.forward()
        
        optimizer.step()
        
        if i % PLOT == 0:
            display.clear_output(wait=True)
            plt.plot(moving_avg(losses))
            plt.show()